In [1]:
import environment
import utility
import os
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
plt.rcParams['figure.figsize'] = [18,12]

### List of principal securities

In [3]:
symbols = ["AAPL","ARQL","CSCO","QQQ","FB","INTC","QCOM","NVDA","CMCSA","AMD","EEM","SPY","GDX","MSFT","AMZN"]
dates = ['01302019','12282018','05302019','05302018','03272019']

#### Function to obtain the series of total Volumes for each hour,second pair
sum over minutes (only execution volume)

In [4]:
def get_total_size(symbol,date):
    a = environment.ts(date,'NASDAQ',symbol,PATH='/Volumes/LaCie/data')
    executions = a.messages.loc[a.messages.type=='E']
    executions.loc[:,'time'] = executions.time.apply(lambda x: utility.parseNanosecondsToDateTime(x))
    executions.loc[:,'hs'] = executions.time.apply(lambda x: (x.hour,x.second))
    ts_total_size_by_sec = executions.groupby(['hs']).mean().loc[:,"size"]
    return ts_total_size_by_sec.loc[ts_total_size_by_sec.index.map(lambda x: x[0]>=9 and x[0]<=16)]

In [ ]:
df = pd.DataFrame()
for s in symbols:
    for d in dates:
        print(s,d)
        try:
            df[s+'_'+d] = get_total_size(s,d)
        except:
            pass

AAPL 01302019


/usr/local/lib/python3.7/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
/usr/local/lib/python3.7/site-packages/pandas/core/indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)


AAPL 12282018
AAPL 05302019
AAPL 05302018
Executing BookConstructor...
pickling...
AAPL 03272019
Executing BookConstructor...


In [ ]:
df.mean(axis=1)
orig_cols = df.columns
df.loc[:,'hour'] = df.index.map(lambda x: x[0])
df.loc[:,'sec'] = df.index.map(lambda x: x[1])

In [ ]:
n=len(set(df.sec.values))
m=len(set(df.hour.values))
B = np.zeros([n,m])
for i in range(n):
    for j in range(m):
        ans = df.loc[(df.hour==j+9) & (df.sec==i)].loc[:,orig_cols].fillna(0).mean(axis=1).values
        if ans.size>0:    
            B[i,j] = ans

A = B/np.sum(B,axis=0)/m
            
vol_df = pd.DataFrame(data=A*100,index = range(n), columns = range(9,9+m))

In [ ]:
sns.heatmap(vol_df)